In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [1]:
def find_max_area(points):
    max_area=0
    for i,pt1 in enumerate(points[:-1]):
        for pt2 in points[i+1:]:
            x1,y1 = pt1
            x2,y2 = pt2
            area = ( abs(x1-x2)+1)*( abs(y1-y2)+1)
            max_area = max(max_area,area)
    return max_area

In [2]:
def part_one(file_path='input_data/test_09.txt'):
    points = []
    with open(file_path,'r') as f:
        for line in f.readlines():
            points.append( [ int(s) for s in line.strip().split(',')  ] )

    area = find_max_area(points)

    return area
           

In [4]:
%%time
part_one()

CPU times: user 834 μs, sys: 1.52 ms, total: 2.36 ms
Wall time: 1.72 ms


50

In [5]:
%%time
part_one('input_data/day_09.txt')
        

CPU times: user 53 ms, sys: 3.1 ms, total: 56 ms
Wall time: 57.7 ms


4771532800

# Part 2


In [ ]:
from collections import defaultdict
import numpy as np

class Interior:

    def __init__(self, points):
        self.points = points
        
        edge_points, min_corner, max_corner = self.find_edges()
        
        self.edge_points = edge_points
        self.min_corner = [ p-1 for p in min_corner]
        self.max_corner = [ p+1 for p in max_corner]
        self.outer_ranges = None
        self.presumed_points = {}
        
    def create_ranges(self, nums):
        nums = list(nums)
        nums.sort()
        ends = np.append(np.where(np.diff(nums) > 1)[0], len(nums)-1)
        starts = np.concat([[0],np.array(np.where(np.diff(nums) > 1)[0])+1])

        ranges=[]
        for s,e in zip(starts,ends):
            ranges.append( (nums[s],nums[e]) )
        return ranges
                
        
    def find_edges(self):

        points = self.points
        edge_points = {'x': defaultdict(list),
                       'y': defaultdict(list),
                      }
    
        min_corner= [float('inf'), float('inf')]
        max_corner= [0,0]
        
        for i,pt1 in enumerate(points):
            pt2= points[(i+1)%len(points)]
    
            min_corner = [ min(min_corner[x], pt1[x]) for x in range(2)]
            max_corner = [ max(max_corner[x], pt1[x]) for x in range(2)]
    
            if pt1[0] == pt2[0]:
                start  = min(pt1[1],pt2[1])
                end  = max(pt1[1],pt2[1])
    
                edge_points['x'][pt1[0]].append( (start,end) ) 
    
            elif pt1[1] == pt2[1]:
                start  = min(pt1[0],pt2[0])
                end  = max(pt1[0],pt2[0]) 
                edge_points['y'][pt1[1]].append( (start,end) ) 
        
        edge_points['x']
        edge_points['y']
        
        return edge_points, min_corner, max_corner

    def check_if_in_edge(self, pt):
        x,y = pt

        for check in self.edge_points['x'][x]:
            if check[0] <= y and check[1] >= y:
                return True
                
        for check in self.edge_points['y'][y]:
            if check[0] <= x and check[1] >= x:
                return True
        return False 
    
    
    def get_neighbors(self, pt):
        lower = self.min_corner
        upper = self.max_corner
        
        neighs = []
        for i in [-1,0,1]:
            for j in [-1,0,1]:
                if i == 0 and j==0:
                    continue
                new = [ pt[0]+i ,  pt[1]+j]
                if new[0] >= lower[0] and new[0] <= upper[0]:
                    if new[1] >= lower[1] and new[1] <= upper[1]:
                        neighs.append(new)
        return neighs
    
    
    def flood_fill_outer(self,) :
        lower = self.min_corner
        upper = self.max_corner
        
        outer_points = {'x': defaultdict(set),
                        'y': defaultdict(set),
                       }
    
        frontier =[lower]
    
        while len(frontier) >0:
            cur = frontier.pop()
            outer_points['x'][cur[0]].add(cur[1])
            outer_points['y'][cur[1]].add(cur[0])
            
    
            for n in self.get_neighbors(cur):
                x,y = n
                if not self.check_if_in_edge(n) and y not in outer_points['x'][x]:
                    
                    frontier.append(n)

        outer_ranges = {'x': {},
                        'y': {},
                       }

        for x,pts in outer_points['x'].items():
            outer_ranges['x'][x] = self.create_ranges(pts)
            
        for y,pts in outer_points['y'].items():
            outer_ranges['y'][y] = self.create_ranges(pts)

        self.outer_ranges = outer_ranges


    def get_sum_to_point(self,pt):
        x,y = pt
        rsum=0
        for tx in range(0,x+1):
            check = np.array(self.outer_ranges['x'][tx])
            rsum+=sum(check <=y)
        return rsum
        
    def presum_squares(self):
        presumed = {}
        for pt in self.points:
            presumed[(x,y)]  = self.get_sum_to_point(pt)

        self.presumed_points = presumed
        
    def check_box_inside(self, pt1, pt2):
        min_corner = [ min(pt2[x], pt1[x]) for x in range(2)]
        max_corner = [ max(pt2[x], pt1[x]) for x in range(2)]

        y_range = ( min_corner[1],max_corner[1])
        x_range = ( min_corner[0],max_corner[0])

        ## Check y edges
        for x in x_range:
            for r in self.outer_ranges['x'][x]:
                if y_range[0] <= r[1] and y_range[1] >= r[0]:
                    return False
        for y in y_range:
            for r in self.outer_ranges['y'][y]:
                if x_range[0] <= r[1] and x_range[1] >= r[0]:
                    return False
        return True
                
    
    def find_max_box(self):
        
        self.flood_fill_outer()
    
        max_area = 0
        
        for i, pt1 in enumerate(self.points):
            for pt2 in self.points[i+1:]:
                if self.check_box_inside(pt1,pt2):
                    x1,y1 = pt1
                    x2,y2 = pt2
                    area = ( abs(x1-x2)+1)*( abs(y1-y2)+1)
                    max_area = max(max_area,area)
                    
        return max_area        
    
        
def part_two(file_path='input_data/test_09.txt'):
    points = []
    with open(file_path,'r') as f:
        for line in f.readlines():
            points.append( [ int(s) for s in line.strip().split(',')  ] )

    check = Interior(points)
    return check.find_max_box()

           

In [52]:
%%time
part_two()

CPU times: user 2.44 ms, sys: 860 μs, total: 3.3 ms
Wall time: 2.52 ms


24

In [ ]:
%%time
part_two('input_data/day_09.txt')

In [5]:
import numpy as np
check = np.array([1,2,3,5,6,7,10,11,12] )

In [27]:
ends = np.append(np.where(np.diff(check) > 1)[0], len(check)-1)
starts = np.concat([[0],np.array(np.where(np.diff(check) > 1)[0])+1])